In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 6.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=7c50c25e73db7cff83cd3a602b96282e6ba5da83fe0db27cfd12708262b4a4a6
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=e8cc46d5dd1fd928d8c74111c238e6bd944864209cf56f7464bfa1dead8fb148
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import tensorflow as tf
import numpy as np




In [3]:
print(tf.__version__)

2.4.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images.shape

(60000, 28, 28)

In [9]:
test_images.shape

(10000, 28, 28)

In [10]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [14]:
def build_model(hp):
  model= keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [17]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [19]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 17s]
val_accuracy: 0.8856666684150696

Best val_accuracy So Far: 0.906499981880188
Total elapsed time: 00h 01m 55s
INFO:tensorflow:Oracle triggered exit


In [20]:
model=tuner_search.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        46144     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                1179680   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 1,226,954
Trainable params: 1,226,954
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1632 - accuracy: 0.9394 - val_loss: 0.2502 - val_accuracy: 0.9138
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1262 - accuracy: 0.9549 - val_loss: 0.2769 - val_accuracy: 0.9147
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0921 - accuracy: 0.9670 - val_loss: 0.2921 - val_accuracy: 0.9135
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0699 - accuracy: 0.9743 - val_loss: 0.3472 - val_accuracy: 0.9048
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0526 - accuracy: 0.9818 - val_loss: 0.3771 - val_accuracy: 0.9143
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0430 - accuracy: 0.9840 - val_loss: 0.3977 - val_accuracy: 0.9110
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0312 - accuracy: 0.9889 - val_loss: 0.4602 - val_accuracy